## Learning a pure advection

To verify that our PINN implementation is working well, we test the PINN on the following problem:
$$
    \begin{cases}
        u_t + au_x = 0\\
        \lim_{|x|\rightarrow\infty}u(t, x) = 0\\
        u(0, x) = u_0(x)
    \end{cases}
$$ where $u_0 = \mathcal{N}(x|0, 1)$. 

Note:

* (07/10/23) Works best when velocity is a constant (i.e. `torch.nn.Parameter([np.random.randn()])`) rather than a spatio-temporally dependent function. Furthermore, should constrain `P_Net()` to output non-negative values, such as using `torch.nn.Softplus` in the very last layer. So far works for identifying any $a \in [0, 1]$.

* (07/12/23) Also works well when the coefficient neural net is only time dependent $V(x)$.

In [ ]:
from PINN.AdvectionNet import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

import numpy as np    
import matplotlib.pyplot as plt

class LinearAdvection1D:
    # Matrix for LA1D 
    A=0
   # Initialization of constants 
    def __init__(self, c, x0, xN, N, deltaT,T):
        self.c = c 
        self.x0 = x0   
        self.xN = xN 
        self.N = N   
        self.deltaT = deltaT   
        self.T = T       
    # CFL number funct.   
    def CFL(self):
        deltaX= (self.xN - self.x0)/self.N
        return np.abs(self.c*self.deltaT/deltaX)
    # check CFL number <=1 or not.
    def checkCFL(self):
        if (self.CFL()<=1):
            flag=True
        else:
            flag=False
        return flag
    # Matrix assembly of LA1D   
    def upwindMatrixAssembly(self):
        alpha_min=min(self.CFL(),0)
        alpha_max=max(self.CFL(),0)
        a1=[alpha_max]*(self.N-1)
        a2=[1+alpha_min-alpha_max]*(self.N)
        a3=[-alpha_min]*(self.N-1)
        self.A=np.diag(a1, -1)+np.diag(a2, 0)+np.diag(a3, 1)

    # Solve u=Au0
    def step(self,u0):
        return np.matmul(self.A,u0) 

#############  
# Start of the code
###################

# constants  
N, x0, xN, deltaT, c, T = 1000, -10.,10., 0.01, 1.0, 10.0
# initialization of constants
LA1D = LinearAdvection1D(c, x0, xN, N, deltaT,T) 

# initial value
x = np.linspace(LA1D.x0,LA1D.xN,LA1D.N)
nx = len(x)
u0=(1/(np.sqrt(2*np.pi)))*np.exp(-0.5*((x+5)**2))

#plot of initial value    
plt.plot(x,u0,label="Initial value")
plt.ylabel('u')
plt.xlabel('x')
plt.legend()

# save solutions
nt = int(LA1D.T/LA1D.deltaT)
tgrid = np.linspace(0, T, nt)
u_sol = np.zeros([nt, nx])


# calculating solution if CFL<=1
if LA1D.checkCFL():
    print("CFL number is: ", LA1D.CFL())
    LA1D.upwindMatrixAssembly()
    for i in range(0, nt):
        u_sol[i, :] = u0
        u = LA1D.step(u0)
        u0 = u
else:
    print("CFL number is greater than 1. CFL: ", LA1D.CFL())

u_sol = u_sol + 0.01 * np.random.randn(u_sol.shape[0], u_sol.shape[1])
# ploting the last solution
plt.plot(x,u_sol[-1, :],label="Solution at t = {}".format(LA1D.T))
plt.legend()
plt.grid(linestyle='dotted')

# save data for testing
scipy.io.savemat("../data/LinearOscillator/Pure_Advection.mat", {
    "xgrid": x.reshape(1, -1),
    "tgrid": tgrid.reshape(1, -1),
    "pmc": u_sol
})

# save a smaller dataset
subsample_t = 2
subsample_x = 5
scipy.io.savemat("../data/LinearOscillator/Pure_Advection_small_data.mat", {
    "xgrid": x.reshape(1, -1)[:, 0:-1:subsample_x],
    "tgrid": tgrid.reshape(1, -1)[:, 0:-1:subsample_t],
    "pmc": u_sol[0:-1:subsample_t, 0:-1:subsample_x]
})
print("Training data size = {}\n".format(u_sol[0:-1:subsample_t, 0:-1:subsample_x].shape))

In [ ]:
# testing PINN # inverse problem with PINN
#data_path = "../data/LinearOscillator/Pure_Advection.mat"
# subsampled spatial training data
data_path = "../data/LinearOscillator/Pure_Advection_small_data.mat"
# create PINN
pinn = AdvectionNet(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR")
# testing training
info = train(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**9,
    epochs=200, batch_print=100, mode="all"
)

In [ ]:
# visualize losses
plt.figure(1, figsize=(16, 8));
plt.plot(np.log10(info["pde_loss"]), lw=2, color="red");
plt.title("PDE Loss vs. Epoch");

plt.figure(2, figsize=(16, 8));
plt.plot(np.log10(info["data_loss"]), lw=2, color="blue");
plt.title("Data Loss vs. Epoch");

# plt.figure(3, figsize=(16, 8));
# plt.plot(np.log10(info["reg_loss"]), lw=2, color="purple");
# plt.title("Regularity Loss vs. Epoch");
query = cartesian_data(pinn.tgrid, pinn.xgrid)

# D_pred = pinn.D_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
# G_pred = pinn.G_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
p_pred = pinn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()

# grids
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

p_exact = pinn.raw_data["pmc"]
dx = pinn.dx.item()
dt = pinn.dt.item()

# plt.figure(1);
# plt.contourf(xgrid, tgrid, G_pred);
# plt.colorbar();

# plt.figure(2);
# plt.plot(xgrid, G_pred[0:-1:10, :].T, alpha=0.4, lw=1.5);
# plt.plot(xgrid, 1.1*np.ones_like(xgrid), "--", lw=4.5, color="red", alpha=0.8, label="exact");
# plt.legend();

In [ ]:
nx = 800
xgrid_fine = np.linspace(-10.0, 10.0, nx)
xgrid_fine = torch.tensor(xgrid_fine)
dx = xgrid_fine[1]-xgrid_fine[0]
nt = 1000
tgrid_fine = np.linspace(0., 10.0, nt)
tgrid_fine = torch.tensor(tgrid_fine)
dt = tgrid_fine[1]-tgrid_fine[0]

query_fine = cartesian_data(tgrid_fine, xgrid_fine)

G_pred_fine = pinn.G_nn(query_fine[:, 1][:, None]).reshape(nx, nt).T.detach().numpy()
p_pred_fine = pinn(query_fine).reshape(nx, nt).T.detach().numpy()

In [ ]:
plt.contourf(G_pred_fine); plt.colorbar();

In [ ]:
plt.figure(1);
plt.plot(xgrid_fine, G_pred_fine[0, :], color="red");
plt.plot(xgrid_fine, np.ones_like(xgrid_fine), "--", color="blue", lw=5.0, alpha=0.5, label="exact");
plt.ylim([0.9, 1.1]);
plt.legend(); 
plt.xlabel(r"$x$");
plt.ylabel(r"$G(x)$");

In [ ]:
# interpolation
print(p_pred_fine.shape)
plt.contourf(xgrid_fine.detach().numpy(), tgrid_fine.detach().numpy(), p_pred_fine);

In [ ]:
# reconstruction on training dataset
print(p_pred.shape)
plt.contourf(pinn.xgrid.detach().numpy(), pinn.tgrid.detach().numpy(), p_pred);

In [ ]:
plt.contourf(pinn.xgrid.detach().numpy(), pinn.tgrid.detach().numpy(), pinn.raw_data["pmc"]);

In [ ]:
plt.figure(2);
# relative error in L^2 from KDE benchmark
l2_rel_err = np.sum(((p_pred-p_exact)**2)*dx, 1) / np.sum((p_exact**2)*dx, 1)
tgrid = pinn.tgrid.detach().numpy()
plt.plot(tgrid, np.log(l2_rel_err), color="red", lw=1.5);
plt.grid(True);
plt.title(r"Relative error in $L^2$");
plt.xlabel(r"$t$"); 
plt.ylabel(r"$L^2(\mathbb{R})$ error");